In [ ]:
%%bash
# preamble script to check and install AMUSE components if necessary

# required packages for this tutorial:
PACKAGES="amuse-framework"
# skip in case a full development install is present
pip show amuse-devel && exit 0
for package in ${PACKAGES} 
do
  pip show ${package} || pip install ${package}
done

In [ ]:
# the following fixes are highly recommended

#allow oversubscription for openMPI
import os
os.environ["OMPI_MCA_rmaps_base_oversubscribe"]="true"

# use lower cpu resources for idle codes
from amuse.support import options
options.GlobalOptions.instance().override_value_for_option("polling_interval_in_milliseconds", 10)


In [ ]:
%matplotlib inline
from matplotlib import pyplot
import numpy

In amuse we mostly work with one or multiple collections of particles. These collections can be thought of as tables were each particle is represented by a row in the table:

<table>
    <tr>
        <th>Particle</th>
        <th>mass</th>
        <th>radius</th>
    </tr>
    <tr>
        <td>1</td>
        <td>10.0</td>
        <td>3.5</td>
    </tr>
    <tr>
        <td>2</td>
        <td>4.0</td>
        <td>1</td>
    </tr>
</table>


<p style="background-color: lightyellow">
<em>Background:</em> AMUSE is optimized to work with columns in the particle collections, each column represents an attribute of the particles in the collection (in the above table the particle collection stores the masses and raddii of the particles). Instead of looping through the particle set we run a function on one or more columns of the set. These functions are often numpy functions and optimized in C, so much faster than looping in python. This will take some time to get used to but often results in more compact Python code that will be easier to understand.
</p>



In [ ]:
from amuse.lab import *

If you know how many particles you want, you can create a collection of particles by specifying the size of the collection. AMUSE will create a set of particles were each particle has a unique 128-bit key. Except for the key, the particles will not have any attributes.

In [ ]:
planets = Particles(7)
print(planets)

The `planets` collection is not very useful yet, it only contains a set of empty particles. We can make it more interesting by specifying a mass and radius.

In [ ]:
planets.mass = [641.85, 4868.5, 5973.6, 102430, 86832, 568460, 1898600] | (1e21 * units.kg)
planets.radius =  [0.532, 0.950, 1, 3.86, 3.98, 9.14, 10.97] | (6384 * units.km)
print(planets)

The above example shows one of the dynamic properties of a particle collection, you can define a new attribute for all particles by assigning a value to the an attribute name. AMUSE does not limit the names, except these have to be valid python attribute names. 

It is easy to specify the same value for all attributes: 

In [ ]:
planets.density = 1000.0 | units.kg / units.m**3
print(planets)

Or request the value of an attribute for all particles:

In [ ]:
print(planets.mass)

We can calculate the density instead of just setting to the same value for all particles.

In [ ]:
planets.volume = 4.0 / 3.0 * numpy.pi * planets.radius**3
planets.density = planets.mass /  planets.volume
print(planets)

If you request an attribute of a particle collection, AMUSE will return a vector quantity. You can do several operations on these vectors:

In [ ]:
print("Total mass of the planets:", planets.mass.sum())
print("Mean density of the planets:", planets.density.mean())

Ofcourse, you can also work with one particle in the set. This works the same as it does for python lists, but instead of an object stored in the list you will get a Particle object that points to the correct row in the particle collection. All changes made on the particle will be reflected in the collection.

In [ ]:
earth = planets[2]
print(earth)

earth.density = 5.52 | units.g / units.cm**3
print(planets)

As the particle is just a pointer into the particle collection, adding a new attribute to a particle will also add a new attribute to the collection, AMUSE will set the value of this new attribute to zero (0.0) for all other particles in the set

In [ ]:
earth.population = 6973738433

print(planets)

Finally, you can also create single particles and add these to a particle collection. (A single particle created like this points to a particle collection with only one particle in it).

In [ ]:
pluto = Particle(mass=1.305 | units.kg, radius=1153 | units.km)
print(pluto)
planets.add_particle(pluto)
print(planets)

A particle collection can represent sets of many different kinds of astrophysical bodies (planets, stars, dark matter, smoothed hydrodynamics particles, etc.). The type of particles in a collection is determined by the attributes (stars may have different attributes than planets) and how you use the set.
Putting different kinds of particles in a set is possible, but in those cases some attributes will have valid values and some will be zero (for example if you would add the sun with it's luminocity to this this table. In practice, we recommend you put one kind of particle in a set (for example have a different set for stars, gas clouds and dark matter particles)

In [ ]:
sun = Particle(mass=1 | units.MSun, radius=1 | units.RSun, luminosity=1 | units.LSun)
planets.add_particle(sun)
print(planets)